In [ ]:
import math
import random

In [ ]:
def distL2(x1,y1, x2,y2):
    xdiff = x2 - x1
    ydiff = y2 - y1
    return int(math.sqrt(xdiff*xdiff + ydiff*ydiff) + .5)

In [ ]:
def distL1(x1,y1, x2,y2):
    return int(abs(x2-x1) + abs(y2-y1)+.5)

In [ ]:
def calc_matriz(coord, dist):
    n = len(coord)
    D = {}
    for i in range(n-1):
        for j in range(i+1,n):
            (x1,y1) = coord[i]
            (x2,y2) = coord[j]
            D[i,j] = dist(x1,y1,x2,y2)
            D[j,i] = D[i,j]
    return n,D

In [ ]:
def read_tsplib(filename):
    f = open(filename, 'r');

    line = f.readline()
    while line.find("EDGE_WEIGHT_TYPE") == -1:
        line = f.readline()

    if line.find("EUC_2D") != -1:
        dist = distL2
    elif line.find("MAN_2D") != -1:
        dist = distL1
    else:
        print("cannot deal with non-euclidean or non-manhattan distances")
        raise Exception

    while line.find("NODE_COORD_SECTION") == -1:
        line = f.readline()

    xy_positions = []
    while 1:
        line = f.readline()
        if line.find("EOF") != -1: break
        (i,x,y) = line.split()

        x = float(x)
        y = float(y)
        xy_positions.append((x,y))

    n,D = calc_matriz(xy_positions, dist)
    return n, xy_positions, D

In [ ]:
def custo(tour):
    z = D[tour[-1], tour[0]]
    for i in range(1,len(tour)):
        z += D[tour[i], tour[i-1]]
    return z,

In [ ]:
def randtour(n):
    sol = list(range(n))
    random.shuffle(sol)
    return sol


In [ ]:
dataset = 'a280.tsp'
path = './'+dataset
print(path)

n, coord, D = read_tsplib(path)

./a280.tsp


In [ ]:
print("Solução aletória:")
tour = randtour(n)
z = custo(tour)
print(f'Rota: {tour}\ncusto: {z}')

Solução aletória:
Rota: [213, 154, 4, 63, 224, 93, 80, 159, 7, 135, 163, 153, 227, 148, 167, 203, 232, 54, 266, 235, 181, 239, 279, 133, 218, 41, 42, 142, 164, 191, 190, 120, 45, 123, 179, 91, 254, 222, 130, 5, 152, 94, 248, 201, 160, 107, 52, 125, 274, 255, 124, 208, 271, 37, 29, 174, 101, 195, 70, 98, 90, 168, 10, 183, 23, 180, 92, 84, 128, 209, 67, 68, 113, 48, 169, 18, 236, 131, 111, 121, 0, 277, 194, 71, 59, 210, 51, 231, 36, 197, 247, 126, 269, 82, 212, 95, 30, 88, 157, 47, 259, 217, 25, 100, 221, 99, 188, 122, 75, 141, 109, 252, 55, 87, 134, 21, 8, 61, 65, 234, 204, 237, 264, 244, 3, 275, 151, 40, 137, 19, 276, 278, 50, 250, 155, 115, 243, 118, 117, 182, 105, 110, 228, 185, 162, 103, 156, 200, 206, 178, 26, 216, 64, 73, 31, 173, 186, 230, 240, 39, 119, 258, 199, 272, 251, 32, 170, 108, 114, 161, 143, 89, 62, 223, 49, 56, 150, 261, 53, 260, 241, 102, 78, 11, 226, 35, 267, 144, 16, 116, 166, 96, 28, 246, 202, 233, 60, 81, 127, 97, 256, 38, 245, 184, 85, 33, 27, 229, 196, 22, 6, 24

In [ ]:
!pip install deap

In [ ]:
import random
import numpy
from deap import base
from deap import creator
from deap import algorithms
from deap import tools

In [ ]:
toolbox = base.Toolbox()

creator.create("Minimizacao", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.Minimizacao)

toolbox.register("aleatorios", random.sample, range(n), n)
toolbox.register("gene", tools.initIterate, creator.Individual, toolbox.aleatorios)
toolbox.register("population", tools.initRepeat, list, toolbox.gene)

toolbox.register("evaluate", custo)
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)


In [ ]:
def statistics(individual):
  return individual.fitness.values

In [ ]:
vezes = 10

for n in range(vezes):

    populacao = toolbox.population(100)

    prob_cross = 0.8
    prob_mut = 0.2
    geracoes = 100


    stats = tools.Statistics(statistics)
    stats.register("média", numpy.mean)
    stats.register("desvio padrão", numpy.std)

    populacao, info = algorithms.eaSimple(
        populacao,
        toolbox,
        prob_cross,
        prob_mut,
        geracoes,
        stats,
        verbose=False
)

    melhores = tools.selBest(populacao, 1)

    for individuo in melhores:
        print(individuo)
        print("Melhor Solução:", individuo.fitness)
        print("--------------------------------------")
        estatisticas = stats.compile(populacao)
        print(estatisticas)

[135, 153, 273, 220, 274, 134, 240, 246, 245, 219, 226, 249, 102, 56, 30, 5, 216, 277, 264, 233, 272, 205, 229, 182, 29, 25, 14, 253, 48, 22, 24, 11, 13, 235, 211, 208, 190, 142, 200, 21, 31, 158, 40, 152, 250, 243, 0, 279, 2, 278, 266, 84, 116, 176, 75, 38, 16, 8, 139, 19, 145, 261, 57, 67, 45, 53, 66, 68, 234, 239, 39, 107, 165, 89, 137, 224, 217, 258, 255, 214, 175, 265, 58, 61, 94, 180, 252, 203, 80, 60, 210, 230, 169, 93, 103, 70, 92, 63, 59, 138, 185, 181, 184, 174, 197, 26, 4, 1, 267, 160, 178, 35, 114, 99, 111, 166, 195, 204, 215, 71, 95, 115, 46, 47, 9, 154, 28, 136, 110, 50, 156, 199, 191, 127, 269, 227, 271, 276, 3, 23, 268, 144, 101, 192, 187, 98, 171, 64, 133, 149, 179, 124, 97, 72, 209, 198, 222, 223, 206, 237, 218, 212, 131, 88, 85, 121, 37, 49, 130, 188, 100, 221, 275, 248, 256, 196, 172, 113, 79, 81, 167, 117, 104, 65, 108, 106, 90, 168, 91, 236, 150, 148, 260, 231, 238, 140, 270, 86, 225, 207, 55, 62, 173, 105, 164, 228, 146, 132, 257, 213, 161, 183, 163, 128, 44, 77,